<a href="https://colab.research.google.com/github/nidaa-hub/HaifaUniv_AI_Lab_2022/blob/main/AI_2022_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***README***

to run this program please change the parameters in the main function (__name__ == "__main__")

comment out the functions/methodes that you want to use 

1.   remove comment out (#) from the function/methode that you want to use (in exampl: (calcPopulationFitness(population) or bullsEye(population))
2.   in line 222 "*mate*" choose the selection methode (Tournement/RWS/etc..) and (single/two/unified)CrossOver
1.   Press ▶ or ctrl+Enter



*   ID Submetter1: 316151232
*   ID Submetter2: 038163630

In [1]:
#///////////////////////Part 1///////////////////////
import time
import math
import numpy as np
import pdb
import operator
from __future__ import division, print_function
from random import randint, random
import sys

CRED = '\033[91m'
CGREEN = '\033[92m'
CFLASH = '\033[5m'
CBOLD = '\33[1m'
CITALIC = '\33[3m'
CEND = '\033[0m'

GA_ELITRATE = 0.1 # elitism rate
GA_MAXITER = 16384 # maximum iterations
GA_POPSIZE = 1000 # ga population size
GA_TARGET = "Hello World!"
GA_MUTATIONRATE = 0.25 #  mutation rate


class Agent:
    def __init__(self, string, fitness):
        self.string = string
        self.fitness = fitness
        self.age = 0

'''def roulette_selection(weights):
      #  performs weighted selection or roulette wheel selection on a list
        and returns the index selected from the list

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value '''

def initPop(population, buffer):
    #initialize population
    for i in range(GA_POPSIZE):
        randString = "".join([chr(randint(32, 122)) for x in range(len(GA_TARGET))])
        population.append(Agent(randString, 0))
        buffer.append(Agent("", 0))

def calcPopulationFitness(population):
    #calculate fitness (letter distance)
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGET)):
            fitness += abs(ord(i.string[j]) - ord(GA_TARGET[j]))
        i.fitness = fitness

def calcAverage(population):
 # calculating the Average and Standard Deviation
    sum_1=0
    for i in population:
        sum_1+= i.fitness

    avg = sum_1/len(population)
    
    sum_2=0
    for i in population:
        sum_2+= math.pow(i.fitness-avg,2)

    s_deviation= math.sqrt(sum_2/len(population))

    print(CEND + "Average:" + CGREEN, avg, CEND + "Standard Deviation:" + CGREEN, s_deviation)

def singlePointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos = randint(0, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos] + population[i2].string[spos:]
    return buffer

def twoPointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos1 = randint(0, target_size - 2)
        spos2 = randint(spos1+1, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos1] + population[i2].string[spos1:spos2] + population[i1].string[spos2:]
    return buffer

def uniformCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        uni = randint(1, 2)
        if uni==1:
          buffer[i].string = population[i1].string
        else:
          buffer[i].string = population[i2].string
    return buffer
    
def elitism(population, buffer, esize):
    return [Agent(x.string, x.fitness) for x in population[:esize]] + [x for x in buffer[esize:]]

def mutate(agent):
    target_size = len(GA_TARGET)
    ipos = randint(0, target_size - 1)
    delta = randint(32, 122)
    agent.string = agent.string[:ipos] + chr((ord(agent.string[ipos]) + delta) % 122) + agent.string[ipos + 1 : ]

def mate(population, buffer, choiceFunc, crossOverFunc):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    target_size = len(GA_TARGET)

    buffer = choiceFunc(population, buffer, esize)
    buffer = crossOverFunc(population, buffer, esize, len(GA_TARGET))

    # Mate the rest
    for i in range(esize, GA_POPSIZE):
        if random() < GA_MUTATIONRATE:
            mutate(buffer[i])
    return buffer

def tournamentSelection(population, buffer, size):
    Selections = []         # array of selections
    for i in range(size):
        pick1 = population[randint(0,GA_POPSIZE-1)]     # pick first genom
        pick2 = population[randint(0,GA_POPSIZE-1)]     # pick second genom

        if pick1.fitness < pick2.fitness:   # find better genom
            Selections.append(Agent(pick1.string, pick1.fitness))
        else:
            Selections.append(Agent(pick2.string, pick2.fitness))

    return Selections + [i for i in buffer[size:]]


def SUS(population, buffer, size):
    #single spin of Roulette wheel
    selections = []
    fitnesses = [(1/agent.fitness) for agent in population]
    index = roulette_selection(fitnesses)
    selections.append(population[index])

    return selections      

def RWS(population, buffer, size):
    #Roulette wheel selection
    selections = []
    fit = [(1/agent.fitness) for agent in population]
    for i in range(size):
        index = roulette_selection(fit)
        selections.append(population[index])

    return selections + [i for i in buffer[size:]] 

def ranking(population, buffer, size):
  # Get the number of individuals in the population.
  #size
  # Use the gauss formula to get the sum of all ranks (sum of integers 1 to N).
  rank_sum = size * (size + 1) / 2
  # Sort and go through all individual fitnesses; enumerate ranks from 1.
  for rank, ind_fitness in enumerate(sorted(population), 1):
    yield rank, ind_fitness, float(rank) / rank_sum  
  # Read the fitnesses from the command line arguments.
        #population.sort(key= lambda x: x.fitness)  # sort population array by fitness
  # Iterate through the computed tuples and print the table rows.

def aging(population, buffer, size):
    adultAge = 6
    for agent in population:
        agent.fitness += pow((agent.age-adultAge),2)   #adding age bonus
    return RWS(population, buffer, size)

def bullsEye(population):
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGET)):
          #if it's bullsEye 
            if i.string[j] == GA_TARGET[j]:  
                fitness+=0
              #if it's not bullsEye but the letter is correct 
            elif i.string[j] in GA_TARGET: 
                fitness+=30
                # if the letter is not correct 
            else:               
                fitness+=80
                
        i.fitness = fitness        
    

if __name__ == "__main__":
    starting_time = time.time()
    population = []
    buffer = []
    #values = []
    initPop(population, buffer)

    for i in range(GA_MAXITER):
        int_clock_ticks = time.time()
        
        #calcPopulationFitness(population)
        bullsEye(population)
        population.sort(key= lambda x: x.fitness)  # sort population array by fitness
        print(CBOLD + "Best:" + CGREEN, population[0].string,CEND + "fitness: " + CGREEN,population[0].fitness)     # print string with best fitness
        calcAverage(population)
        #values[0]= population[0].fitness

        if population[0].fitness==0:
            print(CEND+ "Clock ticks time: " + CGREEN, time.time()-int_clock_ticks)
            print("")
            break
        
        buffer = mate(population, buffer, tournamentSelection, uniformCrossOver)  # mate
        population, buffer = buffer, population

        for agent in population:
            agent.age+=1
            
        print(CEND + "Clock ticks time: " + CRED, time.time()-int_clock_ticks)
        print(""+ CEND)

    #x = np.quantile(values, [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

    #print(x)

    print(CBOLD + "Total time Elapsed: " + CGREEN, time.time()-starting_time)

Best:  T+VHfWgA[!W fitness:  680
Average: 895.26 Standard Deviation: 57.113329442433304
Clock ticks time:  0.012075185775756836

Best: HeHJeBoREp\& fitness:  650
Average: 853.52 Standard Deviation: 50.36675093749842
Clock ticks time:  0.017688512802124023

Best: !l!l\86v/a*o fitness:  680
Average: 816.6 Standard Deviation: 46.82990497534726
Clock ticks time:  0.012518644332885742

Best: !l!l\86vra*o fitness:  600
Average: 783.43 Standard Deviation: 40.41454070009952
Clock ticks time:  0.012053489685058594

Best: !l!l\86vra*o fitness:  600
Average: 755.06 Standard Deviation: 37.89718195327993
Clock ticks time:  0.010817766189575195

Best: !l!l\[6vrX*o fitness:  600
Average: 732.67 Standard Deviation: 34.893711467827686
Clock ticks time:  0.011262893676757812

Best: !l!l\86vra*o fitness:  600
Average: 710.62 Standard Deviation: 30.802850517444227
Clock ticks time:  0.014972448348999023

Best: !l!l\86veaoo fitness:  580
Average: 690.83 Standard Deviation: 25.965575287291205
Clock ticks ti

In [ ]:
#///////////////////////Part 2///////////////////////
from __future__ import division
import time
import math
from random import randint, random
import numpy as np
import pdb
import operator

CRED = '\033[91m'
CGREEN = '\033[92m'
CFLASH = '\033[5m'
CBOLD = '\33[1m'
CITALIC = '\33[3m'
CEND = '\033[0m'

GA_ELITRATE = 0.1
GA_MAXITER = 16384
GA_POPSIZE = 2048
NUM_QUEENS = 8
GA_MUTATIONRATE = 0.75

class Agent:
    def __init__(self, string, fitness):
        self.string = string
        self.fitness = fitness
        self.age = 0

def roulette_selection(weights):
        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value

def initPop(population, buffer):
    ''' initialize population '''
    for i in range(GA_POPSIZE):
        randArray = [randint(0,NUM_QUEENS-1) for i in range(NUM_QUEENS)]  # random numbers array
        
        population.append(Agent(randArray, 0))
        buffer.append(Agent("", 0))

def calcAverage(population):
 # calculating the Average and Standard Deviation
    sum_1=0
    for i in population:
        sum_1+= i.fitness

    avg = sum_1/len(population)
    
    sum_2=0
    for i in population:
        sum_2+= math.pow(i.fitness-avg,2)

    s_deviation= math.sqrt(sum_2/len(population))

    print(CEND + "Average:" + CGREEN, avg, CEND + "Standard Deviation:" + CGREEN, s_deviation)

def getFitness(population):
    #nummber of queens that collide
    for agent in population:
        count = [0 for i in range(NUM_QUEENS)]
        badCount = 0
        for j in agent.string:  #count collision rows
            count[int(j)] +=1
        for x in count:
            if x!=1:    #if collides
                badCount+=1
        for i in range(NUM_QUEENS):
            for j in range(NUM_QUEENS):
                if i==j:
                    continue
                if abs(i-j)== abs(int(agent.string[i]) - int(agent.string[j])): 
                    badCount+=1
                    
        agent.fitness = badCount
            

def singlePointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos = randint(0, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos] + population[i2].string[spos:]
    return buffer

def twoPointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos1 = randint(0, target_size - 2)
        spos2 = randint(spos1+1, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos1] + population[i2].string[spos1:spos2]+ population[i1].string[spos2:]
    return buffer

    
def elitism(population, buffer, esize):
    return [Agent(x.string, x.fitness) for x in population[:esize]] + [x for x in buffer[esize:]]

def mutate(agent):
    target_size = NUM_QUEENS
    ipos = randint(0, target_size - 1)
    delta = randint(0, NUM_QUEENS-1)
    agent.string = agent.string[:ipos] + [(agent.string[ipos] + delta) % NUM_QUEENS] + agent.string[ipos + 1 : ]

def mate(population, buffer, choiceFunc, crossOverFunc):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    target_size = NUM_QUEENS

    buffer = choiceFunc(population, buffer, esize)
    buffer = crossOverFunc(population, buffer, esize, NUM_QUEENS)

    # Mate the rest
    for i in range(esize, GA_POPSIZE):
        if random() < GA_MUTATIONRATE:
            mutate(buffer[i])
    return buffer

def miniConf(soln, nr, iters=1000):
	def posRand(li, filt):
		return random.choice([i for i in range(nr) if filt(li[i])])

	for k in range(iters):
		confs = find_conflicts(soln, nr)
		if sum(confs) == 0:
			return soln
		col = posRand(confs, lambda elt: elt > 0)
		vconfs = [hits(soln, nr, col, row) for row in range(nr)]
		soln[col] = posRand(vconfs, lambda elt: elt == min(vconfs))

def find_conflicts(soln, nr):
	return [hits(soln, nr, col, soln[col]) for col in range(nr)]

def hits(soln, nr, col, row):
	total = 0
	for i in range(nr):
		if i == col:
			continue
		if soln[i] == row or abs(i - col) == abs(soln[i] - row):
			total += 1
	return total

def SUS(population, buffer, size):
    #single spin of Roulette wheel
    selections = []
    fitnesses = [(1/agent.fitness) for agent in population]
    index = roulette_selection(fitnesses)
    selections.append(population[index])

    return selections 

def RWS(population, buffer, size):
    #Roulette wheel selection
    selections = []
    fitnesses = [(1/agent.fitness) for agent in population]
    for i in range(size):
        index = roulette_selection(fitnesses)
        selections.append(population[index])

    return selections + [i for i in buffer[size:]]

def tournamentSelection(population, buffer, size):
    Selections = []   
    for i in range(size):
        pick_1 = population[randint(0,GA_POPSIZE-1)]     #choosing first genom
        pick_2 = population[randint(0,GA_POPSIZE-1)]     #choosing second genom

        if pick_1.fitness < pick_2.fitness:   #selecting better genom
            Selections.append(Agent(pick_1.string, pick_1.fitness))
        else:
            Selections.append(Agent(pick_2.string, pick_2.fitness))

    return Selections + [i for i in buffer[size:]]    

def aging(population, buffer, size):
    aduiltAge = 6
    for agent in population:
        agent.fitness += pow((agent.age-aduiltAge),2)   #adding age bonus
    return RWS(population, buffer, size)

def bullsEye(population):
    for i in population:
        fitness = 0
        for j in range(len(NUM_QUEENS)):
          #if it's bullsEye 
            if i.string[j] == targetString[j]:  
                fitness+=0
              #if it's not bullsEye but the letter is correct 
            elif i.string[j] in targetString: 
                fitness+=30
                # if the letter is not correct 
            else:               
                fitness+=80
                
        i.fitness = fitness              
    

if __name__ == "__main__":
    startingTime = time.time()
    population = []
    buffer = []
    initPop(population, buffer)

    for i in range(GA_MAXITER):
        iterStartingTime = time.time()
        
        getFitness(population)
        population.sort(key= lambda x: x.fitness)  # sort population array by fitness
        print(CBOLD + "Best:" + CGREEN, population[0].string,CEND + "fitness: " + CGREEN,population[0].fitness)     # print string with best fitness
        calcAverage(population)

        if population[0].fitness==0:
            print(CEND+ "Clock ticks time: " + CGREEN, time.time()-int_clock_ticks)
            print("")
            break
        
        
        buffer = mate(population, buffer, tournamentSelection, twoPointCrossOver)  # mate
        population, buffer = buffer, population

        for agent in population:
            agent.age+=1
            
        print(CEND + "Clock ticks time: " + CRED, time.time()-int_clock_ticks)
        print(""+ CEND)

    print(CBOLD + "Total time Elapsed: " + CGREEN, time.time()-starting_time)

Best: [6, 0, 5, 7, 1, 3, 7, 4] fitness:  4
Average: 13.513671875 Standard Deviation: 3.9177092659274737
Clock ticks time:  623.5083229541779

Best: [3, 0, 4, 7, 0, 6, 1, 5] fitness:  2
Average: 11.96240234375 Standard Deviation: 3.2279983170681033
Clock ticks time:  623.6151843070984

Best: [0, 6, 3, 6, 2, 7, 4, 1] fitness:  4
Average: 11.35498046875 Standard Deviation: 3.144076865401994
Clock ticks time:  623.7347438335419

Best: [3, 7, 0, 2, 6, 1, 1, 4] fitness:  2
Average: 11.1513671875 Standard Deviation: 3.292916475109616
Clock ticks time:  623.8571515083313

Best: [3, 7, 0, 2, 6, 1, 1, 4] fitness:  2
Average: 10.79931640625 Standard Deviation: 3.1048191374779925
Clock ticks time:  623.9702982902527

Best: [3, 7, 0, 2, 6, 1, 1, 4] fitness:  2
Average: 10.67138671875 Standard Deviation: 3.056582553299699
Clock ticks time:  624.0964946746826

Best: [3, 6, 4, 1, 5, 0, 5, 7] fitness:  2
Average: 10.5185546875 Standard Deviation: 3.1420857083427527
Clock ticks time:  624.2111051082611
